In [101]:
import numpy as np
n = 3
ezsudoku=np.array([[6, 8, 0, 2, 0, 0, 1, 0, 0],
       [9, 0, 0, 6, 8, 7, 0, 0, 0],
       [0, 2, 0, 9, 0, 0, 0, 6, 8],
       [0, 0, 0, 0, 9, 0, 4, 0, 3],
       [3, 0, 9, 0, 0, 6, 0, 5, 1],
       [0, 0, 0, 3, 0, 5, 9, 2, 0],
       [0, 9, 8, 1, 0, 0, 7, 0, 4],
       [1, 0, 2, 4, 7, 0, 6, 0, 0],
       [7, 0, 4, 5, 0, 8, 0, 0, 0]])
ezsudoku_sol = np.array(
[[6, 8, 3, 2, 5, 4, 1, 9, 7],
[9, 1, 5, 6, 8, 7, 3, 4, 2],
[4, 2, 7, 9, 1, 3, 5, 6, 8],
[2, 5, 6, 8, 9, 1, 4, 7, 3],
[3, 4, 9, 7, 2, 6, 8, 5, 1],
[8, 7, 1, 3, 4, 5, 9, 2, 6],
[5, 9, 8, 1, 6, 2, 7, 3, 4],
[1, 3, 2, 4, 7, 9, 6, 8, 5],
[7, 6, 4, 5, 3, 8, 2, 1, 9]])

In [102]:
np.array(eval(s.replace(" ", ", ").replace("]\n", "],\n")))

array([[6, 8, 0, 2, 0, 0, 1, 0, 0],
       [9, 0, 0, 6, 8, 7, 0, 0, 0],
       [0, 2, 0, 9, 0, 0, 0, 6, 8],
       [0, 0, 0, 0, 9, 0, 4, 0, 3],
       [3, 0, 9, 0, 0, 6, 0, 5, 1],
       [0, 0, 0, 3, 0, 5, 9, 2, 0],
       [0, 9, 8, 1, 0, 0, 7, 0, 4],
       [1, 0, 2, 4, 7, 0, 6, 0, 0],
       [7, 0, 4, 5, 0, 8, 0, 0, 0]])

In [145]:
def convert_to_sparse(onehot):
    idx = np.where(onehot)
    out = np.zeros((n**2, n**2), dtype = int)
    out[idx[:2]] = idx[2] + 1
    return out

def solver(partial_sudoku):
    n = int(np.sqrt(len(partial_sudoku)))
    sol = np.zeros((n**2, n**2, n**2, 2), dtype = int)
    done = sol[:, :, :, 1]
    in_progress = sol[:, :, :, 0]

    indices = tuple(np.meshgrid(np.arange(n**2), np.arange(n**2), indexing="ij"))
    roll_transform = lambda i, j: (i//n*n+j//n, i%n*n+j%n)
    roller = roll_transform(*indices)
    # roller = indices[0]//n*n+indices[1]//n, indices[0]%n*n+indices[1]%n
    transpose = indices[::-1]
    where_initial_onehot = np.where(partial_sudoku != 0)
    where_initial_onehot = tuple([*where_initial_onehot, partial_sudoku[where_initial_onehot]-1])
    done[where_initial_onehot] = 1
    idx = np.where((done.sum(axis = 0, keepdims = True) + done.sum(axis = 1, keepdims = True) + done.sum(axis = 2, keepdims = True) +
                    done[roller].sum(axis = 1, keepdims = True).repeat(n**2, axis = 1)[roller]) == 0)
    in_progress[idx] = 1

    def global_declare_done():
        nonlocal in_progress
        if done.sum() == n**4:
            # print("DONE!")
            return
        n_possible_per_cell = in_progress.sum(axis = 2)
        cells_to_update = np.where(n_possible_per_cell == 1)
        numbers_to_update = np.where(in_progress[cells_to_update] == 1)[1]
        final_coordinates = tuple([*cells_to_update, numbers_to_update])

        if len(final_coordinates[0]) > 0:
        # if True:
            in_progress[final_coordinates] = 0

            in_progress[:, cells_to_update[1], numbers_to_update] = 0
            in_progress[cells_to_update[0], :, numbers_to_update] = 0
            rolled_cells_to_update = roll_transform(*cells_to_update)
            in_progress = in_progress[roller]
            in_progress[rolled_cells_to_update[0], :, numbers_to_update] = 0
            in_progress = in_progress[roller]
            done[final_coordinates] = 1
            global_declare_done()
    global_declare_done()
    return convert_to_sparse(done)




In [146]:
%%timeit
solver(ezsudoku)

801 µs ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [123]:
n**4

81

In [126]:
global_declare_done()
done.sum()

DONE!


81

In [127]:
convert_to_sparse(done)

array([[6, 8, 3, 2, 5, 4, 1, 9, 7],
       [9, 1, 5, 6, 8, 7, 3, 4, 2],
       [4, 2, 7, 9, 1, 3, 5, 6, 8],
       [2, 5, 6, 8, 9, 1, 4, 7, 3],
       [3, 4, 9, 7, 2, 6, 8, 5, 1],
       [8, 7, 1, 3, 4, 5, 9, 2, 6],
       [5, 9, 8, 1, 6, 2, 7, 3, 4],
       [1, 3, 2, 4, 7, 9, 6, 8, 5],
       [7, 6, 4, 5, 3, 8, 2, 1, 9]])

In [128]:
convert_to_sparse(done) == ezsudoku_sol

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True]])

In [55]:
done.sum()

65

In [56]:
done[:, :, 8]

array([[0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [21]:
ezsudoku

array([[6, 8, 0, 2, 0, 0, 1, 0, 0],
       [0, 2, 0, 9, 0, 0, 0, 6, 8],
       [0, 0, 0, 0, 9, 0, 4, 0, 3],
       [3, 0, 9, 0, 0, 6, 0, 5, 1],
       [9, 0, 0, 6, 8, 7, 0, 0, 0],
       [0, 0, 0, 3, 0, 5, 9, 2, 0],
       [0, 9, 8, 1, 0, 0, 7, 0, 4],
       [1, 0, 2, 4, 7, 0, 6, 0, 0],
       [7, 0, 4, 5, 0, 8, 0, 0, 0]])

In [57]:
idx = np.where(done)

In [58]:
final = np.zeros((n**2, n**2))

In [59]:
final[idx[:2]] = idx[2] + 1

In [63]:
final

array([[6., 8., 0., 2., 5., 4., 1., 9., 9.],
       [4., 2., 0., 9., 3., 3., 5., 6., 8.],
       [5., 0., 0., 8., 9., 1., 4., 7., 3.],
       [3., 0., 9., 0., 0., 6., 8., 5., 1.],
       [9., 0., 0., 6., 8., 7., 3., 4., 0.],
       [8., 0., 0., 3., 0., 5., 9., 2., 0.],
       [5., 9., 8., 1., 6., 2., 7., 3., 4.],
       [1., 3., 2., 4., 7., 9., 6., 8., 5.],
       [7., 6., 4., 5., 0., 8., 2., 1., 0.]])

In [65]:
ezsudoku_sol

array([[6, 8, 3, 2, 5, 4, 1, 9, 7],
       [9, 1, 5, 6, 8, 7, 3, 4, 2],
       [4, 2, 7, 9, 1, 3, 5, 6, 8],
       [2, 5, 6, 8, 9, 1, 4, 7, 3],
       [3, 4, 9, 7, 2, 6, 8, 5, 1],
       [8, 7, 1, 3, 4, 5, 9, 2, 6],
       [5, 9, 8, 1, 6, 2, 7, 3, 4],
       [1, 3, 2, 4, 7, 9, 6, 8, 5],
       [7, 6, 4, 5, 3, 8, 2, 1, 9]])

In [64]:
(final == ezsudoku_sol)

array([[ True,  True, False,  True,  True,  True,  True,  True, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False],
       [ True, False, False,  True, False,  True,  True,  True, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True, False,  True,  True,  True, False]])

In [11]:
done.sum(axis = 2)

array([[1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 0, 1],
       [1, 1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 1, 1]])

In [10]:
in_progress.sum(axis = 2)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [7]:
global_declare_done()

In [8]:
done.sum()

68

In [106]:
a = np.array([1,2,3])

In [108]:
a[np.where(np.array([0,0]))] = 100

In [109]:
a

array([1, 2, 3])

In [103]:
np.where(in_progress[np.where(in_progress.sum(axis = 2) == 1)])[1]

array([3, 7, 3, 6, 2, 8, 8, 8], dtype=int64)

In [98]:
in_progress.sum(axis = 2).shape

(9, 9)

In [97]:
np.where(in_progress.sum(axis = 2) == 1)

(array([3, 3, 5, 5, 7, 7, 8, 8], dtype=int64),
 array([1, 6, 0, 8, 5, 8, 7, 8], dtype=int64))

In [94]:
in_progress.sum(axis = 2)

array([[0, 0, 2, 0, 3, 2, 0, 2, 2],
       [2, 0, 3, 0, 4, 3, 0, 0, 0],
       [2, 3, 3, 0, 0, 2, 0, 2, 0],
       [0, 1, 0, 0, 2, 0, 1, 0, 0],
       [0, 4, 3, 0, 0, 0, 0, 0, 0],
       [1, 3, 2, 0, 3, 0, 0, 0, 1],
       [2, 0, 0, 0, 4, 2, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 2, 1],
       [0, 3, 0, 0, 4, 0, 0, 1, 1]])

In [78]:
sol[:, :, :, 1].sum(axis = 2)

array([[1, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 0, 1, 0, 1, 1],
       [1, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1],
       [1, 0, 1, 1, 1, 0, 1, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 0]])

In [43]:
where_initial_onehot

(array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5,
        5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8], dtype=int64),
 array([0, 1, 3, 6, 1, 3, 7, 8, 4, 6, 8, 0, 2, 5, 7, 8, 0, 3, 4, 5, 3, 5,
        6, 7, 1, 2, 3, 6, 8, 0, 2, 3, 4, 6, 0, 2, 3, 5], dtype=int64))

In [41]:
solver(partial_sudoku=ezsudoku)

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
sol.sum(axis = 2) = np.ones((n**2, n**2))